In [15]:
from datasets import load_dataset

# Load the WikiANN dataset
dataset = load_dataset("wikiann", "en")


Found cached dataset wikiann (/home/s6amalia/.cache/huggingface/datasets/wikiann/en/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e)


  0%|          | 0/3 [00:00<?, ?it/s]

In [81]:
dataset = dataset.rename_column("ner_tags", "labels")
dataset = dataset.rename_column("tokens", "text")

In [87]:
from transformers import XLMRobertaTokenizer

# Load XLM-Roberta tokenizer
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

def tokenize_batch(batch):
    return tokenizer(batch["text"],  is_split_into_words=True,truncation=True)

tokenized_datasets = dataset.map(tokenize_batch, batched=True)
# tokenized_datasets = ds.map(tokenize_data, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [88]:
from transformers import XLMRobertaForTokenClassification

# Load pre-trained XLM-Roberta model for token classification
model = XLMRobertaForTokenClassification.from_pretrained("xlm-roberta-base", num_labels=7)
from transformers import TrainingArguments, Trainer

from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions, references=labels)

# Define training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    output_dir="./wikiann_fine_tuned",
    save_total_limit=2,
    num_train_epochs=2,
    logging_dir="./logs",
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics = compute_metrics,
)

# Fine-tune the model
trainer.train()

print(eval_results)

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss


KeyboardInterrupt: 